In [1]:
import numpy as np
from PIL import Image
import ctypes
import matplotlib.pyplot as plt
import random
import os

In [2]:
directory_inputs = ["../database/resized_dataset/28x28_dataset/train_dataset/second_dataset/angkor","../database/resized_dataset/28x28_dataset/train_dataset/second_dataset/parthenon","../database/resized_dataset/28x28_dataset/train_dataset/second_dataset/chateau_bodiam"]

directory_test_inputs = ["../database/resized_dataset/28x28_dataset/test_dataset/second_dataset/angkor","../database/resized_dataset/28x28_dataset/test_dataset/second_dataset/parthenon","../database/resized_dataset/28x28_dataset/test_dataset/second_dataset/chateau_bodiam"]
# directory_inputs = ["../database/resized_dataset/amerique_sud","../database/resized_dataset/asie_sud_est","../database/resized_dataset/rome_grece"]

In [3]:
X = []
Y = []

for i in range(len(directory_inputs)):
    for image_file in os.listdir(directory_inputs[i]):
        if 'image' in image_file:
            if i == 0 :
                image_file = Image.open(directory_inputs[i]+'/'+image_file)
                # print(image_file)
                X.append(np.asarray(image_file))
                Y.append([1., -1., -1.])
            elif i == 1 :
                image_file = Image.open(directory_inputs[i]+'/'+image_file)
                # print(image_file)
                X.append(np.asarray(image_file))
                Y.append([-1., 1., -1.])
            elif i == 2 :
                image_file = Image.open(directory_inputs[i]+'/'+image_file)
                # print(image_file)
                X.append(np.asarray(image_file))
                Y.append([-1.,-1.,1.])

X = np.array(X, dtype=ctypes.c_float)
X_flat = X.flatten()/255 * 2 - 1

Y = np.array(Y, dtype=ctypes.c_float)
Y_flat = Y.flatten()

In [4]:
X_train = []
Y_train = []

for i in range(len(directory_test_inputs)):
    for image_file in os.listdir(directory_test_inputs[i]):
        if 'image' in image_file:
            if i == 0 :
                image_file = Image.open(directory_test_inputs[i]+'/'+image_file)
                # print(image_file)
                X_train.append(np.asarray(image_file))
                Y_train.append([1., -1.,-1.])
            elif i == 1 :
                image_file = Image.open(directory_test_inputs[i]+'/'+image_file)
                # print(image_file)
                X_train.append(np.asarray(image_file))
                Y_train.append([-1., 1.,-1.])
            elif i == 2 :
                image_file = Image.open(directory_test_inputs[i]+'/'+image_file)
                # print(image_file)
                X_train.append(np.asarray(image_file))
                Y_train.append([-1., -1.,1.])

X_train = np.array(X_train, dtype=ctypes.c_float)
X_train_flat = X_train.flatten()/255 * 2 - 1

Y_train = np.array(Y_train, dtype=ctypes.c_float)
Y_train_flat = Y_train.flatten()

In [5]:
my_lib = ctypes.CDLL(r"..\rust_lib\target\release\rust_lib.dll")

my_lib.delete_float_array.argtypes = [
    ctypes.POINTER(ctypes.c_float),
    ctypes.c_int32,
]

my_lib.generate_multi_layer_perceptron_model.argtypes = [ctypes.POINTER(ctypes.c_float), ctypes.c_int32]
my_lib.generate_multi_layer_perceptron_model.restype = ctypes.POINTER(ctypes.c_float)

my_lib.delete_float_array.restype = None

layers = np.array([len(X_flat)/len(X), 8, 8, 3], dtype=ctypes.c_float)
LP_c_float = ctypes.POINTER(ctypes.c_float)
layers_pmc = (ctypes.c_float * len(layers))(*layers)
layers_ptr_pmc = ctypes.cast(layers_pmc, LP_c_float)

print(len(layers_pmc))
number_of_layers = len(layers_pmc)
print(layers)
number_of_classes_to_predict = int(layers[-1])

4
[2352.    8.    8.    3.]


In [6]:
w_array_ptr = my_lib.generate_multi_layer_perceptron_model(layers_ptr_pmc, number_of_layers)
w_array = np.ctypeslib.as_array(w_array_ptr, (2353*8 + 9*8 + 9*3,)) #3*16 + 17*16 + 17*3

LP_c_float = ctypes.POINTER(ctypes.c_float)
x_to_predict = (ctypes.c_float * len(X_flat))(*X_flat)
x_to_predict_ptr = ctypes.cast(x_to_predict, LP_c_float)
LP_c_float = ctypes.POINTER(ctypes.c_float)
y_to_predict = (ctypes.c_float * len(Y_flat))(*Y_flat)
y_to_predict_ptr = ctypes.cast(y_to_predict, LP_c_float)

LP_c_float = ctypes.POINTER(ctypes.c_float)
x_to_predict_test = (ctypes.c_float * len(X_train_flat))(*X_train_flat)
x_to_predict_test_ptr = ctypes.cast(x_to_predict_test, LP_c_float)
LP_c_float = ctypes.POINTER(ctypes.c_float)
y_to_predict_test = (ctypes.c_float * len(Y_train_flat))(*Y_train_flat)
y_to_predict_test_ptr = ctypes.cast(y_to_predict_test, LP_c_float)

my_lib.train_multi_layer_perceptron_model.argtypes = [
    ctypes.POINTER(ctypes.c_float),
    ctypes.POINTER(ctypes.c_float),
    ctypes.c_int32,
    ctypes.POINTER(ctypes.c_float),
    ctypes.c_int32,
    ctypes.POINTER(ctypes.c_float),
    ctypes.c_int32,
    ctypes.c_int32,
    ctypes.POINTER(ctypes.c_float),
    ctypes.POINTER(ctypes.c_float),
    ctypes.c_int32,
    ctypes.c_float,
    ctypes.c_int32,
    ctypes.c_int32,
    ctypes.c_bool,
]
my_lib.train_multi_layer_perceptron_model.restype = ctypes.POINTER(ctypes.c_float)

In [7]:
pmc_trained_model_accuracy_ptr = my_lib.train_multi_layer_perceptron_model(
    w_array_ptr,
    layers_ptr_pmc,  # pointeur vers la forme de pmc
    number_of_layers,  # number of layers
    x_to_predict_ptr,
    len(X),  # number of inputs
    x_to_predict_test_ptr,
    len(X_train),
    int(len(X_flat)/len(X)),  # dimension of inputs
    y_to_predict_ptr,
    y_to_predict_test_ptr,
    3,  # number of classes to predict
    0.0001,  # learning rate
    5,  # epochs
    1,
    True,
)

pmc_trained_model = np.ctypeslib.as_array(pmc_trained_model_accuracy_ptr, (2353*8 + 9*8 + 9*3,))

In [8]:
image_a_predire_bodiam = np.asarray(Image.open('test_images/predict_bodiam.png').resize((28,28)).convert('RGB'))
image_a_predire_angkor = np.asarray(Image.open('test_images/predict_angkor_3.png').resize((28,28)).convert('RGB'))
image_a_predire_parthenon = np.asarray(Image.open('test_images/predict_parthenon.png').resize((28,28)).convert('RGB'))


image_a_predire_data = np.array([image_a_predire_parthenon,image_a_predire_angkor,image_a_predire_bodiam],dtype=ctypes.c_float)
image_a_predire_data_flat=image_a_predire_data.flatten()

In [9]:
LP_c_float = ctypes.POINTER(ctypes.c_float)
x_to_predict_test = (ctypes.c_float * len(image_a_predire_data_flat))(*image_a_predire_data_flat)
x_to_predict_test_ptr = ctypes.cast(x_to_predict_test, LP_c_float)

In [10]:
my_lib.predict_with_multi_layer_perceptron_model.argtypes = [
    ctypes.POINTER(ctypes.c_float),
    ctypes.POINTER(ctypes.c_float),
    ctypes.c_int32,
    ctypes.POINTER(ctypes.c_float),
    ctypes.c_int32,
    ctypes.c_int32,
    ctypes.c_int32,
    ctypes.c_bool,
]

In [11]:
my_lib.predict_with_multi_layer_perceptron_model.restype = ctypes.POINTER(ctypes.c_float)

pmc_predict_ptr = my_lib.predict_with_multi_layer_perceptron_model(
    pmc_trained_model_accuracy_ptr,  # pointeur vers le modele entrainé
    layers_ptr_pmc,  # pointeur vers la forme de pmc
    number_of_layers,  # number of layers
    x_to_predict_test_ptr,  # pointeur vers le dataset à prédire
    len(image_a_predire_data),  # nombre d'éléments du dataset
    int(len(image_a_predire_data_flat)/len(image_a_predire_data)),  # dimension des éléments du dataset
    3,  # number of classes to predict
    True,  # is classification
)


In [19]:
pmc_predict_model = np.ctypeslib.as_array(pmc_predict_ptr, (3 * len(image_a_predire_data),))

In [20]:
for j in range(0,len(pmc_predict_model)-2,3) :
    if pmc_predict_model[j] > pmc_predict_model[j+1] and pmc_predict_model[j] > pmc_predict_model[j+2] :
        print("L'image a deviner vient angkor")
    elif pmc_predict_model[j+1] > pmc_predict_model[j] and pmc_predict_model[j+1] > pmc_predict_model[j+2] :
        print("L'image a deviner vient parthénon")
    elif pmc_predict_model[j+2] > pmc_predict_model[j] and pmc_predict_model[j+2] > pmc_predict_model[j+1]:
        print("L'image a deviner vient bodiam")
    else :
        print('résultats non concluants')

L'image a deviner vient bodiam
L'image a deviner vient bodiam
L'image a deviner vient bodiam


In [21]:
print(len(pmc_predict_model))

9
